In [1]:
from elasticsearch import Elasticsearch

In [10]:
from elasticsearch import Elasticsearch
import redis
import json
import hashlib
import time
from typing import Dict, Any, Optional, List, Tuple

class QueryCache:
    def __init__(
        self,
        redis_host: str = "localhost",
        redis_port: int = 6379,
        es_host: str = "localhost",
        es_port: int = 9200,
        es_index: str = "query_cache",
        similarity_threshold: float = 0.7,
        redis_ttl: int = 86400  # 24 hours in seconds
    ):
        # Initialize Redis client
        self.redis_client = redis.Redis(host=redis_host, port=redis_port, decode_responses=True)
        
        # Initialize Elasticsearch client
        self.es_client = Elasticsearch([f"http://{es_host}:{es_port}"])
        self.es_index = es_index
        self.similarity_threshold = similarity_threshold
        self.redis_ttl = redis_ttl
        
        # Create Elasticsearch index if it doesn't exist
        self._create_es_index_if_not_exists()
    
    def _create_es_index_if_not_exists(self) -> None:
        """Create Elasticsearch index with BM25 settings if it doesn't exist."""
        if not self.es_client.indices.exists(index=self.es_index):
            # Define index settings with BM25 similarity
            settings = {
                "settings": {
                    "number_of_shards": 1,
                    "number_of_replicas": 0,
                    "similarity": {
                        "default": {
                            "type": "BM25",
                            "k1": 1.2,
                            "b": 0.75
                        }
                    }
                },
                "mappings": {
                    "properties": {
                        "query": {
                            "type": "text",
                            "analyzer": "standard"
                        },
                        "query_hash": {
                            "type": "keyword"
                        },
                        "response": {
                            "type": "text" 
                        },
                        "timestamp": {
                            "type": "date"
                        }
                    }
                }
            }
            
            self.es_client.indices.create(index=self.es_index, body=settings)
            print(f"Created Elasticsearch index '{self.es_index}'")
    
    def _generate_query_hash(self, query: str) -> str:
        """Generate a unique hash for the query."""
        return hashlib.md5(query.encode()).hexdigest()
    
    def get_response(self, query: str) -> Tuple[str, bool]:
        """
        Get response for a query from cache or search.
        
        Returns:
            Tuple[str, bool]: (response, cache_hit_flag)
        """
        query_hash = self._generate_query_hash(query)
        
        # Step 1: Check Redis first for exact match
        redis_result = self.redis_client.get(query_hash)
        if redis_result:
            print("Exact match found in Redis cache")
            return json.loads(redis_result), True
        
        # Step 2: Check Redis for similar queries
        similar_query = self._find_similar_in_redis(query)
        if similar_query:
            print(f"Similar query found in Redis: {similar_query['query']}")
            return similar_query['response'], True
        
        # Step 3: Search Elasticsearch with BM25
        es_result = self._search_elasticsearch(query)
        if es_result:
            print(f"Similar query found in Elasticsearch: {es_result['query']}")
            
            # Store in Redis for faster retrieval next time
            self._store_in_redis(query_hash, es_result)
            
            return es_result['response'], True
        
        # No cache hit
        return "", False
    
    def _find_similar_in_redis(self, query: str) -> Optional[Dict[str, Any]]:
        """
        Find similar queries in Redis using a simple lookup strategy.
        In a production system, this would use more sophisticated methods.
        """
        # For simplicity, we're just checking exact keys in Redis
        # In a real system, you might use Redis search capabilities or other approaches
        query_hash = self._generate_query_hash(query)
        if query_hash in self.redis_client.keys():
            response = self.redis_client.get(query_hash)
        else:
            return None
    
    def _search_elasticsearch(self, query: str) -> Optional[Dict[str, Any]]:
        """Search Elasticsearch for similar queries using BM25."""
        search_body = {
            "query": {
                "match": {
                    "query": {
                        "query": query,
                        "operator": "and",
                        "minimum_should_match": "70%"  # Adjust as needed
                    }
                }
            },
            "_source": ["query", "response", "query_hash"],
            "size": 1
        }
        
        try:
            result = self.es_client.search(index=self.es_index, body=search_body)
            hits = result.get('hits', {}).get('hits', [])
            
            if hits and hits[0]['_score'] > self.similarity_threshold:
                source = hits[0]['_source']
                return {
                    'query': source['query'],
                    'response': source['response'],
                    'query_hash': source['query_hash']
                }
        except Exception as e:
            print(f"Error searching Elasticsearch: {e}")
        
        return None
    
    def store_response(self, query: str, response: str) -> None:
        """Store a query and its response in both Redis and Elasticsearch."""
        query_hash = self._generate_query_hash(query)
        
        # Store in Redis
        self._store_in_redis(query_hash, {
            'query': query,
            'response': response,
            'query_hash': query_hash
        })
        
        # Store in Elasticsearch
        self._store_in_elasticsearch(query, response, query_hash)
        
        print(f"Stored new query-response pair in cache: {query[:30]}...")
    
    def _store_in_redis(self, query_hash: str, data: Dict[str, Any]) -> None:
        """Store data in Redis with TTL."""
        self.redis_client.setex(
            query_hash,
            self.redis_ttl,
            json.dumps({
                'query': data['query'],
                'response': data['response'],
                'query_hash': data['query_hash']
            })
        )
    
    def _store_in_elasticsearch(self, query: str, response: str, query_hash: str) -> None:
        """Store query and response in Elasticsearch."""
        document = {
            'query': query,
            'response': response,
            'query_hash': query_hash,
            'timestamp': int(time.time() * 1000)  # Current time in milliseconds
        }
        
        try:
            self.es_client.index(index=self.es_index, body=document)
        except Exception as e:
            print(f"Error storing in Elasticsearch: {e}")


# Example usage
if __name__ == "__main__":
    # Initialize the cache
    cache = QueryCache()
    
    # Example query
    user_query = "Quang Trung sinh ngày bao nhiêu"
    
    # Try to get response from cache
    response, cache_hit = cache.get_response(user_query)
    
    if cache_hit:
        print(f"Cache hit! Response: {response}")
    else:
        print("Cache miss! Generating new response...")
        # In a real system, this would call your backend service or AI model
        new_response = "5 tháng 11 năm 1753"
        
        # Store the new response in cache
        cache.store_response(user_query, new_response)
        response = new_response
    
    print(f"Final response: {response}")

Exact match found in Redis cache
Cache hit! Response: {'query': 'Ngày sinh của vua Quang Trung ngày bao nhiêu?', 'response': '5 tháng 11 năm 1753', 'query_hash': '98acbdf15edd032c5c32647592aa109e'}
Final response: {'query': 'Ngày sinh của vua Quang Trung ngày bao nhiêu?', 'response': '5 tháng 11 năm 1753', 'query_hash': '98acbdf15edd032c5c32647592aa109e'}
